1RSB
===

\begin{align*}
p\left(f\right)  &= \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y F_{ai}}\\
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1,\dots,d}f_b\right)e^{-y F_{ia}}\\
\end{align*}

where
\begin{align*}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
 F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}



In [221]:
using OffsetArrays

const ∏ = prod
const ∑ = sum


function iter_slow_factor(Q, n, y=0.0)
    k = length(Q)
    p = OffsetVector(zeros(2N+1), -N:N)
    for fs in Iterators.product(fill(-N:N,k)...)
        Fai = -2*minimum(abs.(fs))*(∏(fs) < 0)
        w = ∏(q[f1] for (q,f1) ∈ zip(Q,fs)) * exp(-y*Fai)
        f = minimum(abs.(fs))*sign(∏(fs))
        p[clamp(f, -N, N)] += w
    end
    p ./ sum(p)
end

function iter_slow_var(P, s, N, y=0.0) 
    q = OffsetVector(zeros(2N+1), -N:N)
    for fs in Iterators.product(fill(-N:N, length(P))...)
        f = sum(fs) + s
        Fia = abs(f) - sum(abs.(fs)) 
        w = ∏(p[f1] for (p,f1) ∈ zip(P,fs)) * exp(-y*Fia)
        q[clamp(f, -N, N)] += w 
    end
    q ./ sum(q)
end

iter_slow_var (generic function with 3 methods)

Simplifications
--

$F_{ai}$ can be simplified: 

\begin{align}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\left|\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= 2f\Theta(-f)
\end{align}

So

\begin{align*}
p\left(f\right)= & \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y 2 f \Theta(-f)}\\
\end{align*}

To compute $p$, define

\begin{align*}
a_k(f) &= p\left(\textrm{sign}\left(\prod_{i=1}^k f_i\right)=\textrm{sign}(f) \wedge |f_1|,\dots,|f_k| \ge |f|\right)\\
\end{align*}

$a_k$ satisfies the recursion

\begin{align*}
a_0(f) &= \delta(\textrm{sign}(f)-1)\\
a_k(f) &= a_{k-1}(f) \sum_{f_k\geq |f|}q_k(f_k) + a_{k-1}(-f) \sum_{f_k \leq -|f|} q_k(f_k)\\
\end{align*}

Then we finally have

$$p_k(f) = \cases{(a_k(f)-a_k(f+\textrm{sign}(f)))e^{-y2f\Theta(-f)} & for $f\neq0$\\
           1-\prod_{j=1}^k(1-q_j(0)) & for $f=0$}$$


In [193]:
function iterffixed(Q, N, y=0)
    a = OffsetVector(zeros(2N+3), -N-1:N+1)
    a[1:lastindex(a)] .= 1
    for q ∈ Q
        for f=1:N+1
            ap, am = a[f], a[-f]
            sp, sm = ∑(q[f:N]), ∑(q[-N:-f])
            a[+f] = ap*sp + am*sm
            a[-f] = am*sp + ap*sm
        end
    end
    p = OffsetVector(zeros(2N+1), -N:N)
    for f = 1:N
        p[+f] = (a[+f]-a[+f+1])
        p[-f] = (a[-f]-a[-f-1])*exp(2y*f)
    end
    p[0] = 1-∏(1-q[0] for q ∈ Q)
    p ./ sum(p)
end


iterffixed (generic function with 2 methods)

Comparison
--

In [244]:
N=10
y=0.1
Q=[(p=OffsetArray(zeros(2N+1),-N:N); p[-9:9] .= rand(19); p ./=sum(p); p) for i=1:3]
[iterffixed(Q,N,1) iter_slow_factor(Q,N,1)]

21×2 Matrix{Float64}:
 0.0          0.0
 0.383062     0.383062
 0.34141      0.34141
 0.189418     0.189418
 0.0717821    0.0717821
 0.0110971    0.0110971
 0.00253552   0.00253552
 0.000566925  0.000566925
 0.000108003  0.000108003
 1.00023e-5   1.00023e-5
 3.30936e-6   3.30936e-6
 1.35452e-6   1.35452e-6
 1.97207e-6   1.97207e-6
 1.41193e-6   1.41193e-6
 8.50097e-7   8.50097e-7
 4.9927e-7    4.9927e-7
 4.4646e-7    4.4646e-7
 1.58689e-7   1.58689e-7
 3.70541e-8   3.70541e-8
 6.64263e-9   6.64263e-9
 0.0          0.0

The expression for $F_{ia}$ is

\begin{align*}
    F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}
and

\begin{align*}
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1}^df_b\right)e^{-y F_{ia}}\\
&= e^{-y |f|}\sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b)e^{y|f_b|} \delta\left(f-s-\sum_{b=1}^df_b\right)\\
\end{align*}


To compute $q_d$, note that

\begin{align}
q(f) &= q_d(f)e^{-y|f|}
\end{align}

where $q_d$ satisfies
\begin{align}
q_0(f) & = \delta(f-s)\\
q_d(f) & = \sum_{f_d} q_{d-1}(f-f_d) p_d(f_d)e^{-y|f_d|} 
\end{align}

In [234]:
function ⊛(p1, p2)
    N = max(lastindex(p1), lastindex(p2))
    q = OffsetArray(zeros(2N+1),-N:N)
    for f1 in eachindex(p1)
        for f2 in eachindex(p2)
            q[clamp(f1+f2, -N, N)] += p1[f1]*p2[f2]
        end
    end
    q
end

function itergfixed(P, s, N, y=0)
    q = OffsetVector(zeros(2N+1), -N:N)
    q[s]=1
    w = exp.(y .* abs.(eachindex(q)))
    for p ∈ P
        q = q ⊛ (p ./ w)
    end
    q ./= w
    q ./ sum(q)
end

itergfixed (generic function with 3 methods)

Comparison
--

In [246]:
N=10
y=0.1
s=1
P=[(p=OffsetArray(zeros(2N+1),-N:N); p[-9:9] .= rand(19); p ./=sum(p); p) for i=1:3]
[itergfixed(P[1:2],s,N,0) iter_slow_var(P[1:2],s,N,0)]

21×2 Matrix{Float64}:
 0.106964   0.106964
 0.0329656  0.0329656
 0.0335879  0.0335879
 0.0354667  0.0354667
 0.032129   0.032129
 0.0346542  0.0346542
 0.0374614  0.0374614
 0.035893   0.035893
 0.037156   0.037156
 0.0405135  0.0405135
 0.0460118  0.0460118
 0.0501356  0.0501356
 0.0534975  0.0534975
 0.0495592  0.0495592
 0.0461773  0.0461773
 0.0499736  0.0499736
 0.0430719  0.0430719
 0.0373767  0.0373767
 0.0234817  0.0234817
 0.0258439  0.0258439
 0.148079   0.148079

In [142]:
using Distributions, ProgressMeter

uni(N) = (p=OffsetVector(ones(2N+1), -N:N); p ./= sum(p))
residual(x) = (p=OffsetVector((x .* eachindex(x))[1:end], 0:lastindex(x)-1); p./=sum(p))

function RSB(Λ, K; maxiter=100, popsize=1000, N=10, popP = [uni(N) for i=1:popsize], popQ = [uni(N) for i=1:popsize], y=0)
    Λ1 = residual(Λ)
    K1 = residual(K)

    distΛ1 = DiscreteNonParametric(collect(eachindex(Λ1)), Λ1.parent)
    distK1 = DiscreteNonParametric(collect(eachindex(K1)), K1.parent)

    @showprogress for t = 1:maxiter
        for i = eachindex(popQ)
            d = rand(distΛ1)
            k = rand(distK1)
            Q = rand(popQ, k)
            P = rand(popP, d)
            popQ[i] = iter_slow_var(P, N, y)
            popP[i] = iter_slow_factor(Q, N, y)
        end
    end
    popP, popQ
end

RSB (generic function with 1 method)

In [143]:
Λ = OffsetVector([0,0,1], 0:2)
K = OffsetVector([0,0,0,1], 0:3)


N=5
popsize=10000
popQ = [OffsetVector(ones(2N+1)/(2N+1),-N:N) for i=1:popsize]
popP = [OffsetVector(ones(2N+1)/(2N+1),-N:N) for i=1:popsize]

10000-element Vector{OffsetVector{Float64, Vector{Float64}}}:
 [0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]
 [0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]
 [0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]
 [0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]
 [0.09

In [144]:
popP, popQ = RSB(Λ,K; N=5, maxiter=100, popsize=popsize, popQ=popQ, popP=popP, y=1.0);

Progress: 100%|█████████████████████████████████████████| Time: 0:06:17


In [145]:
using StatsBase

sort(proportionmap(argmax.(popP)))

OrderedCollections.OrderedDict{Int64, Float64} with 7 entries:
  -3 => 0.0074
  -2 => 0.0998
  -1 => 0.3281
  0  => 0.4504
  1  => 0.1079
  2  => 0.0063
  3  => 0.0001

In [124]:
using SparseArrays
q=OffsetVector(zeros(2N+1),-N:N); q[1]=1

Q=[q for i=1:2]; 

p=iter_slow_factor(Q, N);
@show p'
q1=iter_slow_var([p],N);
@show q1'


p' = [0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0]
q1' = [0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]


1×9 adjoint(OffsetArray(::Vector{Float64}, -4:4)) with eltype Float64 with indices Base.OneTo(1)×OffsetArrays.IdOffsetRange(-4:4):
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0

1RSB equations for Max-Sum (Survey Propagation) at finite parameter $y$
---
We consider the auxiliary statistical physics model
\begin{align}
    P_y(\{u_{ai},u_{ia}\})=\frac{1}{Z(y)}\mathbb{I}\left(\{u_{ai},u_{ia}\}\text{sat Max-Sum}\right)e^{-yF(\{u_{ai},u_{ia}\})}
\end{align}
in which the variables are the Max-Sum messages living on the edges, and the constraints (that can be put in a factorized form) enforces the Max-Sum equations to be satisfied. Each Max-Sum solution represents a cluster. The Max-Sum solutions are weighted according to their Bethe free energy, which corresponds to the minimal energy (inside the cluster). In the $y\to\infty$ limit one keeps only the clusters with minimal energy

* The distributions $Q_{ia}(u_{ia})$, $Q_{ai}(u_{ai})$ over Max-Sum messages obey the 1RSB equations :
\begin{align}
    Q_{ia}(u_{ia})&=\frac{1}{Z_{ia}}\sum_{\{u_{bi}\}_{b\in\partial i\setminus a}}\delta(u_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}))e^{-yC_{ia}}\prod_{b\in\partial i\setminus a}Q_{bi}(u_{bi}) \\
    Q_{ai}(u_{ai})&=\frac{1}{Z_{ai}}\sum_{\{u_{ja}\}_{j\in\partial a\setminus i}}\delta(u_{ai}-f^{MS}_{ai}(\{u_{ja}\}_{j\in\partial a\setminus i}))e^{-yC_{ai}}\prod_{j\in\partial a\setminus i}Q_{ja}(u_{ja})    
\end{align}
where $u_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a})$, and $u_{ai}-f^{MS}_{ai}(\{u_{ja}\}_{j\in\partial a\setminus i})$ are shorthand notation for the Max-Sum equations
\begin{align}
    u_{ia}(\sigma_i)&=s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i) - C_{ia}\\
    u_{ai}(\sigma_i)&=\max_{\{\sigma_j\}_{j\in\partial a\setminus i}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}u_{ja}(\sigma_j)\right) - C_{ai}
\end{align}
and $C_{ia}$, $C_{ai}$ are the constants in the Max-Sum equations
\begin{align}
    C_{ia}&=\max_{\sigma_i}\left(s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i)\right)\\
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}u_{ja}(\sigma_j)\right)
\end{align}
the constants $Z_{ia}$, $Z_{ai}$ are normalizations of the distributions $Q_{ia}$, $Q_{ai}$ over the Max-Sum messages

* With the parametrization $u_{ai} = f_{ai}\sigma_i+g_{ai}$ we get:
\begin{equation}
    g_{ai} = -\frac{1}{\beta}\log(2{\rm cosh}(\beta f_{ai})\to-|f_{ai}|
\end{equation}
Therefore :
\begin{align*}
    C_{ia}&=\max_{\sigma_i}\left(\sigma_i\left(s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right)\right) + \sum_{b\in\partial i\setminus a}g_{bi}\\
    &=\left|s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right| - \sum_{b\in\partial i\setminus a}|f_{bi}|
\end{align*}
and
\begin{align*}
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}f_{ja}\sigma_j\right)-\sum_{j\in\partial a\setminus i}|f_{ja}|\\
    &= -2\min_{j\in\partial a\setminus i}|f_{ja}|\Theta\left(-\prod_{j\in\partial a\setminus j}f_{ja}\right)
\end{align*}

* We can then write the same 1RSB equations for the distributions $P_{ia}(f_{ia})$, $P_{ai}(f_{ai})$ over the integer parameters $f_{ia}$, $f_{ai}$. We can also write a closed equation on the distribution $P_{ai}$, using that :
\begin{equation}
    P_{ai}(f_{ia})=\frac{1}{Z_{ia}}\sum_{\{f_{bj}\}_{j\in\partial a \setminus i, b\in\partial j\setminus a}}\delta(f_{ia}-F^{MS}(\{f_{bj}\})e^{-y\left(C_{ai}+\sum_{j\in\partial a\setminus i}C_{ja}\right)}\prod_{j\in\partial a \setminus i, b\in\partial j\setminus a}P_{bj}(f_{bj})
\end{equation}
with:
\begin{align*}
    C_{ai}+\sum_{j\in\partial a\setminus i}C_{ja} &= \max_{\sigma:sat}\left(\sum_{j\in\partial a \setminus i}\sigma_j\left(s_j + \sum_{b\in\partial j\setminus a}f_{bj}\right)\right)-\sum_{j\in\partial a\setminus i}\sum_{b\in\partial j\setminus a}|f_{bj}|
\end{align*}

* Random graph ensemble
When the source and the factor graph are random variables the 1RSB messages $P_{ai}$, $(a,i)\in E$ become random variables. Let $(a,i)$ be a uniformly chosen edge in a factor graph drawn from the random ensembl, and let $\mathcal{P}$ be the distribution of the message $P_{ai}$ solution of the 1RSB equation written above. The distribution $\mathcal{P}(p)$ obey consistency equation similar to the RS cavity equations:
\begin{align}
    \mathcal{P}(P)=\sum_{k}\tilde{P}_k\sum_{d_1,\dots, d_k}\left(\prod_{i=1}^k\tilde{\Lambda}_{d_i}\right)\int\prod_{i=1}^k\prod_{j=1}^{d_i}{\rm d}\mathcal{P}(P_{ij})\delta(P-F^{1RSB}(\{P_{i,j}\}))
\end{align}
This equation always admits a trivial fixed point $\mathcal{P}(P)=\delta[P-p^{RS}]$, where $p^{RS}$ is the solution of the RS cavity equation.
In the RS phase, this trivial fixed-point is the unique solution, while in the 1RSB phase, the trivial solution becomes unstable and the above equation admits a non-trivial solution.